# Libraries

In [219]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif
import tensorflow as tf
import tensorflow
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import seaborn as sns
import pandas as pd
import numpy as np
np.random.seed(5)
tf.random.set_seed(5)

In [220]:
df = pd.read_csv("dataset/heart.csv")

In [221]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


# Selecting top 10 most Contributing Features

In [222]:
bestfeatures = SelectKBest(score_func=f_classif, k=10)

In [223]:
fit = bestfeatures.fit(df.drop(['target'], axis = 1), df['target'])

In [224]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(df.columns)
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Specs','Score']
print(featureScores.nlargest(10,'Score'))

       Specs       Score
9    oldpeak  243.450780
8      exang  242.883889
2         cp  238.558457
7    thalach  222.799538
11        ca  174.877078
10     slope  138.679393
12      thal  131.802996
1        sex   86.689707
0        age   56.785135
3   trestbps   20.087353


# Splitting the data & Pipelining the model

In [243]:
X = df.drop(['target', 'fbs', 'restecg', 'chol'], axis = 1).values
y = to_categorical(df['target'].values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)

# Taking Two User from test data for finetuning

In [244]:
user_data_X = X_test[0:1]
user_data_y = y_test[0:1]

X_test = np.delete(X_test, [0, 1], 0)
y_test = np.delete(y_test, [0, 1], 0)

# Training the model for the first time

In [245]:
model = Sequential()
model.add(Dense(16, input_dim=10, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, shuffle=True)

Epoch 1/50
29/29 [==============================] - 0s 2ms/step - loss: 26.2877 - accuracy: 0.5119
Epoch 2/50
29/29 [==============================] - 0s 1ms/step - loss: 18.0521 - accuracy: 0.5119
Epoch 3/50
29/29 [==============================] - 0s 1ms/step - loss: 12.9574 - accuracy: 0.5119
Epoch 4/50
29/29 [==============================] - 0s 1ms/step - loss: 8.7556 - accuracy: 0.5119
Epoch 5/50
29/29 [==============================] - 0s 1ms/step - loss: 5.5779 - accuracy: 0.5119
Epoch 6/50
29/29 [==============================] - 0s 1ms/step - loss: 3.1694 - accuracy: 0.5119
Epoch 7/50
29/29 [==============================] - 0s 1ms/step - loss: 1.3886 - accuracy: 0.5347
Epoch 8/50
29/29 [==============================] - 0s 1ms/step - loss: 0.9598 - accuracy: 0.5380
Epoch 9/50
29/29 [==============================] - 0s 2ms/step - loss: 0.8808 - accuracy: 0.5531
Epoch 10/50
29/29 [==============================] - 0s 2ms/step - loss: 0.8181 - accuracy: 0.5868
Epoch 11/50
29/2

In [246]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 6ms/step - loss: 0.4388 - accuracy: 0.8515


[0.43876913189888, 0.8514851331710815]

In [247]:
model.save("Heart_Classifier_NN_v1_orig.keras")

# Finetuning the model with one data

In [248]:
model = tf.keras.models.load_model("Heart_Classifier_NN_v1_orig.keras")
history = model.fit(user_data_X, user_data_y, epochs=50, shuffle=True)

Epoch 1/50
1/1 [==============================] - 0s 373ms/step - loss: 0.3758 - accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 6ms/step - loss: 0.3479 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 10ms/step - loss: 0.3064 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 0.2596 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: 0.2140 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 7ms/step - loss: 0.1732 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 0s/step - loss: 0.1385 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 17ms/step - loss: 0.1102 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 4ms/step - loss: 0.0876 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 11ms/step - loss: 0.0699 - accuracy: 1.0000
Epoch 11/50
1/1 [==================

In [249]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 2.1877 - accuracy: 0.5248


[2.187746524810791, 0.5247524976730347]

# Finetuning the model with new data Merged with original Data

In [250]:
X_train = np.concatenate((user_data_X, X_train), axis = 0)

In [251]:
y_train = np.concatenate((user_data_y, y_train), axis = 0)

In [252]:
model = tf.keras.models.load_model("Heart_Classifier_NN_v1_orig.keras")
history = model.fit(X_train, y_train, epochs=50, shuffle=True)

Epoch 1/50
29/29 [==============================] - 0s 2ms/step - loss: 0.4730 - accuracy: 0.8093
Epoch 2/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4681 - accuracy: 0.8169
Epoch 3/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4708 - accuracy: 0.8126
Epoch 4/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8223
Epoch 5/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4571 - accuracy: 0.8223
Epoch 6/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4551 - accuracy: 0.8267
Epoch 7/50
29/29 [==============================] - 0s 2ms/step - loss: 0.4537 - accuracy: 0.8212
Epoch 8/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4514 - accuracy: 0.8191
Epoch 9/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4476 - accuracy: 0.8245
Epoch 10/50
29/29 [==============================] - 0s 1ms/step - loss: 0.4480 - accuracy: 0.8277
Epoch 11/50
29/29 [

In [253]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 6ms/step - loss: 0.3326 - accuracy: 0.9010


[0.3325673043727875, 0.9009901285171509]

# Point of Doubts
* Retraining with Entire data with new data can cause overfitting, resource consumption
* Retraining with Only new data significantly reduces Accuracy